In [15]:
!pip install scikit-learn


In [16]:
import time
import requests
import pandas as pd
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
import time
from IPython.display import clear_output
import requests
import pandas as pd
import pandas_ta as ta
try:
    import talib
    USE_TALIB = True
except ImportError:
    USE_TALIB = False
from datetime import datetime, date, time as dt_time, timedelta
from zoneinfo import ZoneInfo
from plotly.subplots import make_subplots




import plotly.graph_objects as go

# ── CONFIG ─────────────────────────────────────────────────────────────────
API_KEY = "vBRy5un9PuHfxFj1IrHpfg8a2RS57jE9"

# https://github.com/polygon-io/client-python/blob/master/examples/websocket/stocks-ws.py

In [17]:
def fetch_polygon_data(ticker: str, api_key: str,
                              start_date: date,
                              end_date: date,
                              start_time: dt_time,
                              end_time: dt_time = dt_time(16, 0),
                              multiplier: int = 1,
                              timespan: str = "minute",
                              limit: int = 50000) -> pd.DataFrame:
    """
    Fetches intraday minute-level OHLCV data for a given ticker from Polygon.io.

    Args:
        ticker (str): Stock ticker symbol (e.g., "TSLA").
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch data.
        end_date (date): The date for which to end fetch data.
        start_time (datetime.time): Start of time window in EST.
        end_time (datetime.time): End of time window in EST.
        multiplier (int): Interval multiplier (default 1 minute).
        timespan (str): Time unit ("minute", "hour", etc.).
        limit (int): Maximum number of results to retrieve.

    Returns:
        pd.DataFrame: A filtered DataFrame with renamed columns and timestamps in EST.
    """
    ET_ZONE = ZoneInfo("America/New_York")

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{start_date}/{end_date}"
    params = {
        "apiKey": api_key,
        "adjusted": "true",
        "sort": "asc",
        "limit": limit
    }

    session = requests.Session()
    response = session.get(url, params=params)
    response.raise_for_status()

    bars = response.json().get("results", [])
    df = pd.DataFrame(bars)

    if df.empty:
        return pd.DataFrame()  # Return empty DataFrame if no data

    df["TIME_UTC"] = pd.to_datetime(df["t"], unit="ms", utc=True)
    df["TIME_EST"] = df["TIME_UTC"].dt.tz_convert(ET_ZONE)
    del df["TIME_UTC"]

    # Define time window
    start_dt = datetime.combine(start_date, start_time, tzinfo=ET_ZONE)
    end_dt = datetime.combine(end_date, end_time, tzinfo=ET_ZONE)

    # Filter and rename
    df = df[(df["TIME_EST"] >= start_dt) & (df["TIME_EST"] <= end_dt)].copy()
    df.rename(columns={
        "o": "open",
        "h": "high",
        "l": "low",
        "c": "close",
        "v": "volume",
        "n": "trades",
        "vw": "vwap"
    }, inplace=True)

    return df


In [18]:

def add_candlestick_patterns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds all TA-Lib candlestick pattern columns and a summary column with detected patterns.
    
    Returns:
        DataFrame with candlestick pattern columns and a 'Detected_Patterns' summary column.
    """
    # Build dictionary of candlestick pattern functions
    cdl_patterns = {
        func: getattr(talib, func)
        for func in dir(talib)
        if func.startswith("CDL")
    }

    # Apply each pattern function to the OHLC data
    for name, pattern_func in cdl_patterns.items():
        try:
            df[name] = pattern_func(df['open'], df['high'], df['low'], df['close'])
        except Exception as e:
            print(f"⚠️ Could not compute {name}: {e}")
            df[name] = 0  # Fallback to 0 if something fails

    # Detect and summarize patterns per row
    def detect_patterns(row):
        patterns = []
        for name in cdl_patterns:
            val = row[name]
            if val != 0:
                direction = "Bullish" if val > 0 else "Bearish"
                strength = abs(val)
                patterns.append(f"{name} ({direction}, Strength={strength})")
        return ', '.join(patterns) if patterns else None

    # Add summary column
    df["Detected_Patterns"] = df.apply(detect_patterns, axis=1)

    return df

In [19]:

def add_indicators_and_signals(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds technical indicators, buy conditions, and entry signal column to the given DataFrame.
    Assumes columns: 'open', 'high', 'low', 'close', 'volume'

    Returns:
        DataFrame with new columns added.
    """

    # ----------------- add candlestick patterns -----------------
    df = add_candlestick_patterns(df)
    # ------------------------------------------------------------

    # Bollinger Bands
    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=2, nbdevdn=2)
    df['BB_up_2'], df['BB_dn_2'] = up, dn

    up, _, dn = talib.BBANDS(df.close.values, timeperiod=20, nbdevup=1, nbdevdn=1)
    df['BB_up_1'], df['BB_dn_1'] = up, dn

    # VWAP (Cumulative approximation)
    df["VWAP_Cum"] = (df["close"] * df["volume"]).cumsum() / df["volume"].cumsum()

    # EMA, MACD, ATR, SMA

    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

        # EMA, MACD, ATR, SMA
    df['EMA9'] = talib.EMA(df.close.values, timeperiod=9)
    df['EMA20'] = talib.EMA(df.close.values, timeperiod=20)
    df['EMA50'] = talib.EMA(df.close.values, timeperiod=50)
    df['EMA200'] = talib.EMA(df.close.values, timeperiod=200)



    macd, sig, hist = talib.MACD(df.close.values, fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'], df['Signal'], df['MACD_hist'] = macd, sig, hist

    df['ATR'] = talib.ATR(df.high.values, df.low.values, df.close.values, timeperiod=14)
    df['SMA5'] = talib.SMA(df.close.values, timeperiod=5)
    df['SMA20'] = talib.SMA(df.close.values, timeperiod=20)
    df['SMA50'] = talib.SMA(df.close.values, timeperiod=50)
    df['SMA200'] = talib.SMA(df.close.values, timeperiod=200)

    # Candlestick pattern: Marubozu
    df['CDLMARUBOZU'] = talib.CDLMARUBOZU(df.open.values, df.high.values, df.low.values, df.close.values)
    df['Bullish_marubozu'] = df['CDLMARUBOZU'] == 100
    df['Bearish_marubozu'] = df['CDLMARUBOZU'] == -100

    # RSI
    df['RSI'] = talib.RSI(df.close.values, timeperiod=14)

    # ----------------- Buy Conditions -----------------

    # B1: Price ≥ SMA5
    df['B_1'] = df['close'] >= df['SMA5']

    # B2: MACD histogram increasing & MACD strong
    df['MACD_diff'] = df['MACD'] - df['Signal']
    df['MACD_diff_avg'] = df['MACD_diff'].rolling(15).mean()
    df['MACD_strong'] = df['MACD_diff'] > 2 * df['MACD_diff_avg']
    df['B_2'] = (df['MACD_hist'].diff() > 0) & df['MACD_strong']

    # B3: RSI > smoothed RSI
    df['RSI_smooth'] = df['RSI'].ewm(span=3, adjust=False).mean()
    df['B_3'] = df['RSI'] > df['RSI_smooth']

    # B4: Volume spike
    df['Vol_avg20'] = df['volume'].rolling(20).mean()
    df['B_4'] = df['volume'] > 1.5 * df['Vol_avg20']

    # B5: Avoid bad momentum patterns
    df['Cond_MACD_hist_avoid'] = (df['MACD_hist'] > 0) & (df['MACD_hist'].diff() < 0)
    df['Cond_RSI_falling'] = df['RSI'].diff() < 0
    df['Cond_price_lower'] = df['close'] < df['close'].shift(1)
    df['B_5'] = df[['Cond_MACD_hist_avoid', 'Cond_RSI_falling', 'Cond_price_lower']].any(axis=1)

    # B6: Bullish marubozu breakout above recent high with follow-through
    N = 10
    df['RecentHigh'] = df['high'].shift(1).rolling(N).max()
    df['Breakout_follow_through'] = (
        (df['high'] > df['RecentHigh']) &
        (df['close'] > df['RecentHigh']) &
        (df['close'] > df['open'])
    )
    df['B_6'] = df['Bullish_marubozu'] & df['Breakout_follow_through']

    # Final Entry Signal
    df['Entry_Signal'] = df['B_1'] & df['B_2'] & df['B_3'] & ~df['B_5'] & df['B_6']

    return df

In [20]:
from datetime import date
import time
from typing import List, Dict
import pandas as pd

def fetch_multiple_tickers_data(ticker: List[str],
                                api_key: str,
                                start_date: date,
                                end_date: date,
                                start_time: dt_time,
                                end_time: dt_time,
                                multiplier: int,
                                timespan: str,
                                limit: int ,
                                delay: float = 0.25) -> Dict[str, pd.DataFrame]:
    """
    Fetch minute-level Polygon data for multiple tickers on a given date.

    Args:
        ticker (List[str]): List of stock ticker symbols.
        api_key (str): Your Polygon.io API key.
        start_date (date): The date for which to fetch intraday data.
        delay (float): Seconds to wait between API calls (rate limit buffer).

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of ticker symbol → DataFrame.
    """
    all_data = {}

    for ticker in ticker:
        try:
            df = fetch_polygon_data(
                ticker=ticker,
                api_key=API_KEY,
                start_date=start_date,
                end_date=end_date,
                start_time=start_time,
                end_time=end_time,
                multiplier=multiplier,
                timespan=timespan,
                limit=limit
                )
            if not df.empty:
                
                df = add_indicators_and_signals(df)

                all_data[ticker] = df

            print(f"✅ {ticker} - fetched {len(df)} rows")

        except Exception as e:
            print(f"❌ Error fetching {ticker}: {e}")

        time.sleep(delay) # ⏱️ Avoid hitting rate limits

    return all_data


In [21]:
# Add example 100 tickers

# Add example 100 tickers

TICKERS = ["AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA", "META", "NFLX", "INTC", "AMD", 
           "BA", "CRM", "DIS", "ADBE", "PYPL", "CSCO", "PEP", "KO", "T", "VZ",
           "WMT", "COST", "MCD", "NKE", "QCOM", "ORCL", "IBM", "GE", "CAT", "HON",
           "JNJ", "PFE", "MRK", "LLY", "ABBV", "BMY", "CVX", "XOM", "SLB", "COP",
           "JPM", "BAC", "WFC", "GS", "MS", "BLK", "TGT", "LOW", "HD", "SBUX",
           "UBER", "LYFT", "ABNB", "SHOP", "ROKU", "ZM", "PLTR", "SNOW", "NET",
           "PANW", "CRWD", "ZS", "OKTA", "DDOG", "DOCU", "F", "GM", "RIVN", "LCID",
           "TSM", "ASML", "AVGO", "TXN", "MU", "INTU", "ADSK", "EA", "TTWO",
           "NOK", "ERIC", "V", "MA", "AXP", "PYPL", "SOFI", "HOOD", "COIN",
           "SPOT", "BIDU", "BABA", "JD", "PDD", "TME", "IQ", "YUMC", "NTES", "LI"]


TICKERS = ['HIMS','BABA','NVDA','TSLA','GOOG','JPM','MS','STX','FDX','GS','USB','KEY','CFG','NKE','CMG','COO','XHB','ITB','GDXJ',"SRFM"]

# TICKERS = ["SRFM","PLTR","AAPL", "MSFT", "GOOG", "TSLA", "AMZN", "NVDA"]
# TICKERS = ['BULL']


In [24]:



# Date range 
start_date = date(2025, 8, 25)
end_date = date(2025, 8, 25)
# 12-13 weekend
# Time window for filtering intraday bars (Eastern Time)
start_time = dt_time(0, 0)   # ex.,4:00 AM ET
end_time = dt_time(20, 0)    #ex.,4:00 PM ET
#-------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Granularity of data
multiplier = 1               #   e.g., 1-minute bars,  5-minute bars, etc.
timespan = "minute"          #  "second", "minute", "hour", "day", etc.

# Max number of bars to retrieve per request (Polygon max = 50,000)
limit = 50000

# ⬇️ Fetch intraday data for multiple tickers from Polygon
all_data_minute = fetch_multiple_tickers_data(
    ticker=TICKERS,
    api_key=API_KEY,
    start_date=start_date,
    end_date=end_date,
    start_time=start_time,
    end_time=end_time,
    multiplier=multiplier,
    timespan=timespan,
    limit=limit
)




/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_23922/2719099855.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



✅ HIMS - fetched 183 rows


/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_23922/2719099855.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



✅ BABA - fetched 250 rows


/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_23922/2719099855.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



...

✅ ITB - fetched 22 rows


/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_23922/2719099855.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



✅ GDXJ - fetched 36 rows


/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_23922/2719099855.py:90: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



✅ SRFM - fetched 63 rows


In [ ]:
from IPython.display import display
import pandas as pd

start_end_times = {}

for ticker, df in all_data_minute.items():
    if isinstance(df, pd.DataFrame) and not df.empty and "TIME_EST" in df.columns:
        start_time = df["TIME_EST"].min()
        end_time = df["TIME_EST"].max()
        start_end_times[ticker] = {
            "Start_Time": start_time,
            "End_Time": end_time
        }

# Convert to DataFrame for display
df_time_range = pd.DataFrame.from_dict(start_end_times, orient="index")
df_time_range.sort_values(by="Start_Time", inplace=True)

display(df_time_range)


In [ ]:
def calc_first_buy_last_sell_pct(df,price_col='close',buy_col='buy_condition',sell_col='Sell_condition'):
    
    """
    Calculate the percent change from the first Buy to the last Sell.

    Returns:
        float: Percent gain/loss, or None if no valid buy/sell.
    """
    if not df[buy_col].any() or not df[sell_col].any():
        return None  # No valid buy or no valid sell condition

    first_buy_idx  = df.index[df[buy_col]].min()
    last_sell_idx  = df.index[df[sell_col]].max()

    buy_price  = df.at[first_buy_idx, price_col]
    sell_price = df.at[last_sell_idx, price_col]

    return (sell_price - buy_price) / buy_price * 100

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

def add_tsf_forecast_to_df(df: pd.DataFrame, regression_length: int = 25, forecast_length: int = 5) -> pd.DataFrame:
    """
    Adds a Time Series Forecast (TSF) column to the DataFrame using rolling linear regression.

    Parameters:
    - df: DataFrame with 'close' column
    - regression_length: lookback window for regression
    - forecast_length: how far ahead to forecast

    Returns:
    - DataFrame with new column 'TSF_forecast'
    """
    tsf_values = [np.nan] * len(df)
    X_template = np.arange(regression_length).reshape(-1, 1)

    for i in range(regression_length, len(df) - forecast_length):
        y_window = df['close'].iloc[i - regression_length:i].values
        model = LinearRegression().fit(X_template, y_window)
        # tsf_values[i + forecast_length] = model.predict([[regression_length + forecast_length - 1]])[0]
        tsf_values[i] = model.predict([[regression_length - 1]])[0] # This is modified by farnaz  to aligne the results with


    df['TSF_forecast'] = tsf_values
    return df


from scipy.stats import linregress

def slope(series):
    x = range(len(series))
    slope, _, _, _, _ = linregress(x, series)
    return slope


In [ ]:
import talib
import pandas as pd
import numpy as np
import pandas_ta as ta

# TICKERS = ['BULL']

# ─────────────────────────────────────────────────────────────────────
# Assume all_data_minute is a dict: {"AAPL": df1, "TSLA": df2, ...}
# and TICKERS is your list of symbols
# ─────────────────────────────────────────────────────────────────────
TICKERS_Plot=TICKERS

all_data_minute_processed = {}

for TICKER in TICKERS_Plot:
    df = all_data_minute[TICKER].copy()

    # ─────────────── Indicators Setup ───────────────
    df["ATR_pct"] = df["ATR"] / df["close"]
    df["MACD_amplified"] = (df["MACD"].abs() > 2 * df["Signal"].abs().rolling(window=20).mean())
    df['VWAP_pct_change'] = df['VWAP_Cum'].pct_change()
    df['VWAP_slope'] = df['VWAP_Cum'].rolling(window=5).apply(slope, raw=False)
    df['VWAP_change'] = df['VWAP_Cum'].diff()
    df["MACD_hist_smooth_EMA"] = df["MACD_hist"].ewm(span=5, adjust=False).mean()



    tsi_df = ta.tsi(df['close'], long=25, short=13, signal=8)
    tsi_df.index = df.index  # Align index to match df

    # Rename TSI columns to simpler names
    tsi_df.columns = ['TSI', 'TSI_signal']

    # Join directly
    df = df.join(tsi_df)

    df = add_tsf_forecast_to_df(df, regression_length=25, forecast_length=5)

    # ─────────────── BUY SEGMENT ───────────────
    buy_condition = (
        (df["close"] > df["VWAP_Cum"]) &
        (df["SMA50"] > df["SMA200"]) &
        (df["VWAP_Cum"] > df["SMA200"]) &
        (df["MACD_hist"].diff() > 0) &
        (df["MACD_hist"] > 0) &
        (df["RSI"].diff() > 0) 
        & (df["ATR_pct"] > 0.001) 
        & (df["MACD_amplified"])
    )

    df["buy_condition"] = buy_condition

    # ─────────────── SELL SEGMENT ───────────────
    df['was_above'] = df['close'].shift(1) > df['EMA50'].shift(1)
    df['is_below'] = df['close'] <= df['EMA50']
    df['Sell_condition'] = df['was_above'] & df['is_below']

    # -----Nullify any Sell that happens before the first Buy and after the last buy ------

    if df['buy_condition'].any():

        # ptu Sell as False before the first Buy 
            df.loc[~df['buy_condition'].cummax(), 'Sell_condition'] = False

            # find the index of the last Buy
            last_buy_idx = df.index[df['buy_condition']].max()

            # find the first Sell after last Buy index
            first_sell_idx = df.loc[last_buy_idx + 1:, 'Sell_condition'].idxmax()
            
            # mask all sells, then re-enable only that first one
            df.loc[df.index > first_sell_idx, 'Sell_condition'] = False

    # ─────────────── Save the processed DataFrame ───────────────
    all_data_minute_processed[TICKER] = df
#___________________________________________________________________
tickers_to_buy = []

for TICKER in TICKERS:
    if TICKER not in all_data_minute_processed:
        continue

    df = all_data_minute_processed[TICKER]

    # Confirm it's a DataFrame (not a tuple accidentally)
    if isinstance(df, tuple):
        df = df[0]  # if stored as (df,) in the dictionary

    if "buy_condition" in df.columns and df["buy_condition"].any():
        tickers_to_buy.append(TICKER)

print("✅ Tickers with at least one Buy signal:")
print(tickers_to_buy)


# TICKER= "NVDA" # adx = 38
# df=all_data_minute_processed[TICKER]
# display(df[df["buy_condition"]][['TIME_EST','close', 'VWAP_Cum', 'EMA50', 'EMA200', 'SMA50', 'SMA200', 'ATR_pct', 'ATR']])
# ['MS', 'USB', 'XHB', 'ITB', 'GDXJ', 'SRFM']


In [ ]:
# for stock with out activity in pre or post market apply a time filter or solve it in an other way
# CMG,ITB 7-17
# 7-16,GOOG (macd rising is not met, smooth version need to be used),ITB(sma200 is not riding),NKE, emmidiate trough, vwap>ema200 and sma200 
# vwap>sma20 , <sma50

In [23]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import time
from IPython.display import display, HTML


# TICKERS_Plot=['BULL']
TICKERS_Plot=tickers_to_buy





# Loop over all tickers
for ticker in TICKERS_Plot:

    df=all_data_minute_processed[ticker]
    pct_change = calc_first_buy_last_sell_pct(df)

    # display(HTML(
    #     "<div style='overflow-x:auto'>" +
    #     df[df["buy_condition"]][['TIME_EST', 'close', 'VWAP_Cum', 'EMA50', 'EMA200', 
    #         'SMA50', 'SMA200', 'ATR_pct', 'ATR', 
    #         'VWAP_pct_change', 'VWAP_slope', 'VWAP_change'
    #     ]].to_html(max_rows=30, max_cols=1000) +
    #     "</div>"
    # ))




    fig = make_subplots(
        rows=4, cols=1, 
        shared_xaxes=True,
        specs=[
        [{"secondary_y": True}],  # Row 1
        [{}],                     # Row 2
        [{}],                     # Row 3
        [{}],                     # Row 4
            ],   
        row_heights=[0.3, 0.15, 0.15, 0.08],
        vertical_spacing=0.02,
        subplot_titles=("Price + BB + VWAP", "RSI5", "MACD Hist, MACD & Signal", "TSI") )

    # Row 1: Price + BB + VWAP
    fig.add_trace(go.Candlestick(x=df["TIME_EST"], open=df["open"], high=df["high"],low=df["low"], close=df["close"], name="Price"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_up_1"], line=dict(dash="dash"), name="BB Up 1"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["BB_dn_1"], fill="tonexty", fillcolor="rgba(200,200,200,0.2)", name="BB Dn 1"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["VWAP_Cum"], line=dict(dash="dot"), name="VWAP"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA200"], line=dict(color="orange"), name="EMA200"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA200"], line=dict(color="blue"), name="SMA200"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA50"], line=dict(color="black"), name="SMA50"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["EMA50"], line=dict(color="khaki"), name="EMA50"), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["SMA5"], line=dict(color="gray"), name="SMA5"), row=1, col=1, secondary_y=False)

    # fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["TSF_forecast"], line=dict(color="yellow"), name="TSF_forecast"), row=1, col=1, secondary_y=False)



    # Row 1: Volume zones
    df['volume_color'] = np.where(df['close'] > df['close'].shift(1), 'green',np.where(df['close'] < df['close'].shift(1), 'red', 'gray'))
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["volume"], name="Vol", marker_color=df["volume_color"],width=1000,opacity=.3,marker_line_width=0), row=1, col=1,secondary_y=True)

    # Row 2: RSI5
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["RSI"], name="RSI", line=dict(color="olive")), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", row=2, col=1)
    fig.add_hline(y=50, line_dash="dash", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", row=2, col=1)

    # Row 3: MACD Histogram, MACD & Signal
    hist_colors = ['green' if val >= 0 else 'red' for val in df["MACD_hist"]]
    fig.add_trace(go.Bar(x=df["TIME_EST"], y=df["MACD_hist"],marker=dict(color=hist_colors, line=dict(color=hist_colors, width=1)),name='MACD Hist'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['MACD'], line=dict(color='olive', width=2), name='MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["Signal"], name="Signal", line=dict(color="salmon", width=1)), row=3, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["MACD_hist_smooth_EMA"], name="MACD_hist_smooth_EMA", line=dict(color="brown", width=1)), row=3, col=1)

    # Row 4: TSI
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI'], line=dict(color='red', width=2), name='TSI'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['TSI_signal'], line=dict(color='blue', width=2), name='TSI_signal'), row=4, col=1)

    # fig.add_hline(y=-40, line=dict(dash="dash", width=0.5), row=4, col=1)
    fig.add_hline(y=-25, line=dict(dash="dash", width=0.5), row=4, col=1)
    fig.add_hline(y=0,    line=dict(dash="dash", width=0.5), row=4, col=1)
    fig.add_hline(y=25,   line=dict(dash="dash", width=0.5), row=4, col=1)
    # fig.add_hline(y=40,   line=dict(dash="dash", width=0.5), row=4, col=1)



    # # Row 3: MACD & Signal
    # fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df['MACD'], line=dict(color='olive', width=2), name='MACD'), row=3, col=1)
    # fig.add_trace(go.Scatter(x=df["TIME_EST"], y=df["Signal"], name="Signal", line=dict(color="salmon", width=1)), row=3, col=1)
    # fig.add_hline(y=0,line=dict(color="gray", width=1, dash="dash"),row=3, col=1)



    # Layout and annotations
    fig.update_layout(
        title=f"Intraday ET for {ticker} on {df['TIME_EST'].dt.date.iloc[-1]}" 
              f"({pct_change:+.2f}% from first Buy to last Sell)",
        # annotations=[dict(text=ann, x=0.5, y=1.02, xref="paper", yref="paper", showarrow=False, font_family="monospace")],
        xaxis_rangeslider_visible=False,height=900, width=1000,paper_bgcolor="white", plot_bgcolor="white")

    # Gridlines
    fig.update_xaxes(showgrid=True, gridcolor='lightgray', rangeslider_visible=False)
    fig.update_yaxes(showgrid=True, gridcolor='lightgray')

    # Add vertical dashed green lines at each entry and exit point
    for t in df.loc[df["buy_condition"], "TIME_EST"]:
        fig.add_vline(x=t,line=dict(color='green', width=1, dash='dash') )# row=1, col=1  # Or remove this to extend across all rows

    for t in df.loc[df["Sell_condition"], "TIME_EST"]:
        fig.add_vline(x=t,line=dict(color='red', width=1, dash='dash') )# row=1, col=1  # Or remove this to extend across all rows

    # Show figure
    fig.show()


In [180]:
target_time = "2025-07-09 12:33:00-04:00"
df[df["TIME_EST"] == target_time][['TIME_EST', 'close', 'VWAP_Cum', 'VWAP_pct_change', 'VWAP_slope', 'VWAP_change']]


,TIME_EST,close,VWAP_Cum,VWAP_pct_change,VWAP_slope,VWAP_change
409,2025-07-09 12:33:00-04:00,6.61,7.092626,-0.000017,-0.000155,-0.00012


In [210]:
df = all_data_minute['NVDA'].copy()
start_time = pd.Timestamp("2025-07-07 09:28:00-04:00")
end_time = pd.Timestamp("2025-07-07 09:43:00-04:00")

df_interval = df[(df["TIME_EST"] >= start_time) & (df["TIME_EST"] <= end_time)]
df_interval[["TIME_EST", "MACD_hist"]]


,TIME_EST,MACD_hist
328,2025-07-07 09:28:00-04:00,0.018098
329,2025-07-07 09:29:00-04:00,0.031296
330,2025-07-07 09:30:00-04:00,0.068187
331,2025-07-07 09:31:00-04:00,0.105841
332,2025-07-07 09:32:00-04:00,0.119039
333,2025-07-07 09:33:00-04:00,0.088185
334,2025-07-07 09:34:00-04:00,0.062094
335,2025-07-07 09:35:00-04:00,0.066693
336,2025-07-07 09:36:00-04:00,0.069387
337,2025-07-07 09:37:00-04:00,0.073785


In [ ]:
TICKER= "NVDA" # adx = 38
# TICKER= "GOOG"   # adx = 42
# TICKER= "AMZN" # adx = 23 MACD>
# TICKER= "PLTR" # adx = 28 MACD>
# TICKER= "AVGO"
# TICKER= "SOFI"# adx = 25 MACD>
# TICKER= "HOOD"

# TICKER= "MSFT"
# TICKER= "TSLA"

# Bad
# TICKER= "ROKU" # Sideways Trend end of MACD trend
# TICKER= "LCID" # Sideways Trend
# TICKER= "COIN" # MACD and Signal <0 , EMA 50< EMA 200
# TICKER= "TME" # end of the trend
# TICKER= "IQ"# Sideways Trend, EMA 50 < SMA 50 , MACD and Signal <0 ,

df=all_data_minute[TICKER]
# df=all_data_second[TICKER]


import pandas as pd
import talib

# ─────────────── Indicators Setup ───────────────
df["ATR_pct"] = df["ATR"] / df["close"]
df["MACD_amplified"] = (df["MACD"].abs() > 2 * df["Signal"].abs().rolling(window=20).mean())
df["adx"] = talib.ADX(df["high"], df["low"], df["close"], timeperiod=14)

# ─────────────── BUY SEGMENT ───────────────
buy_condition = (
    (df["SMA50"] > df["SMA200"]) &
    (df["VWAP_Cum"] > df["SMA200"]) &
    (df["MACD_hist"].diff() > 0) &
    (df["MACD_hist"] > 0) &
    (df["RSI"].diff() > 0) &
    (df["ATR_pct"] > 0.001) &
    (df["MACD_amplified"])
)

df["buy_condition"] = buy_condition
df["buy_segment"] = (buy_condition != buy_condition.shift()).cumsum()

# buy_rows = []
# for seg_id, group in df.groupby("buy_segment"):
#     if group["buy_condition"].iloc[0]:  # started with True condition
#         buy_rows.append(group.iloc[0].copy())

# df_buy_signal = pd.DataFrame(buy_rows)
# df_buy_signal["Signal"] = "Buy"
df_buy_signal = df[df['buy_condition']].copy()
#----------------------------------------------------------------------------------



#----------------------------------------------------------------------------------



# # ──────── Detect Cross Below EMA50 ────────
df['was_above'] = df['close'].shift(1) > df['EMA50'].shift(1)
df['is_below'] = df['close'] <= df['EMA50']
df['cross_below'] = df['was_above'] & df['is_below']

# ──────── Filter Cross-Below Events ────────
df_sell_signal = df[df['cross_below']].copy()
# df_sell_signal = df_cross[['TIME_EST', 'close', 'EMA50']]

# ─────────────── Combine and Sort ───────────────
display(df_sell_signal[['close', 'EMA50']])
display(df_buy_signal[['close','VWAP_Cum' , 'EMA50', 'EMA200','SMA50', 'SMA200' , 'ATR_pct','ATR']])


clean_entry_times = df_buy_signal.index
clean_exit_times = df_sell_signal.index

# print(clean_entry_times)
# print(clean_exit_times)



/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_5889/1395072069.py:28: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_5889/1395072069.py:29: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/2g/jw8wtxw938g1b33q2nygph8w0000gn/T/ipykernel_5889/1395072069.py:42: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

,close,EMA50
53,162.800000,162.824939
68,162.820000,162.820531
79,162.800000,162.832158
87,162.790000,162.837147
91,162.830000,162.836482
100,162.840000,162.866562
110,162.880000,162.883925
125,162.820000,162.823745
130,162.800000,162.823077
133,162.810000,162.826408


,close,VWAP_Cum,EMA50,EMA200,SMA50,SMA200,ATR_pct,ATR
322,163.7200,163.154370,163.200733,163.100889,163.171586,163.099177,0.000874,0.143030
324,164.0004,163.587452,163.250151,163.115544,163.202394,163.109179,0.001141,0.187205
325,164.5000,163.725869,163.299164,163.129320,163.230994,163.117579,0.001374,0.225962
326,165.0600,164.091678,163.368217,163.148530,163.271794,163.128829,0.001570,0.259107
329,165.2100,164.317912,163.561086,163.205373,163.393468,163.162448,0.001692,0.279524
336,165.4400,164.534999,163.917936,163.327730,163.658100,163.239253,0.002006,0.331918
337,165.4650,164.579514,163.978605,163.348997,163.703600,163.252728,0.002126,0.351781


In [90]:
display(df_signals_SELL)

# ── PLOTLY SUBPLOTS ───────────────────────────────────────────────────────────
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    row_heights=[0.5,0.2,0.2,0.1],
    vertical_spacing=0,
    subplot_titles=("Price + MA5/EMA9/VWAP/BB","RSI(5)","MACD","Volume")
)

# ─ BB lines + fill (light, and placed first) ────────────────────────────────
fig.add_trace(go.Scatter(
    x=df.index, y=df["BB_up"],
    line=dict(color="gray", dash="dash"),
    name="BB Upper"
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df["BB_dn"],
    fill="tonexty",
    fillcolor="rgba(189,183,107,0.15)",  # much lighter khaki
    line=dict(color="gray", dash="dash"),
    name="BB Lower"
), row=1, col=1)

# ─ Price Candles ─────────────────────────────────────────────────────────────
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"], high=df["high"],
    low=df["low"], close=df["close"],
    increasing_line_color="YellowGreen",
    decreasing_line_color="darkred",
    name="Price"
), row=1, col=1)

# ─ MA/EMA/VWAP ────────────────────────────────────────────────────────────────
fig.add_trace(go.Scatter(x=df.index, y=df["EMA9"],  line=dict(color="red"),     name="EMA9"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA50"], line=dict(color="gold"),        name="EMA50"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA50"], line=dict(color="magenta"),        name="SMA50"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA5"],  line=dict(color="fuchsia"),     name="SMA5"), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["EMA200"], line=dict(color="blue"),        name="EMA200"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["SMA200"], line=dict(color="black"),        name="SMA200"),row=1,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["VWAP_Cum"], line=dict(color="Maroon"),  name="VWAP"),row=1,col=1)

# ─ RSI + Oversold/Overbought Bands ─────────────────────────────────────────────
fig.add_trace(go.Scatter(
    x=df.index, y=df["RSI"], line=dict(color="rebeccapurple"), name="RSI(5)"
), row=2, col=1)
fig.add_hrect(y0=70, y1=100, fillcolor="darkkhaki", opacity=0.3, row=2, col=1)
fig.add_hrect(y0=0,  y1=30,  fillcolor="darkkhaki", opacity=0.3, row=2, col=1)

# ─ MACD & Histogram ──────────────────────────────────────────────────────────
hist_cols = df["MACD_hist"].apply(lambda x: "green" if x>=0 else "red")
fig.add_trace(go.Bar(x=df.index, y=df["MACD_hist"], marker_color=hist_cols, name="MACD Hist"), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["MACD"],   line=dict(color="pink"),  name="MACD"), row=3, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["Signal"], line=dict(color="black"), name="Signal"),row=3,col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["ATR"], line=dict(color="red"), name="ATR"),row=3,col=1)

# ─ Volume Overlay ─────────────────────────────────────────────────────────────
# fig.add_trace(go.Bar(x=df.index, y=df["vol_up"], name="Vol > avg", marker_color="olive",  offsetgroup="vol"), row=4, col=1)
# fig.add_trace(go.Bar(x=df.index, y=df["vol_dn"], name="Vol ≤ avg", marker_color="red",   offsetgroup="vol"), row=4, col=1)

# ─ ADDED BY FARNAZ ─────────────────────────────────────────────────────────────
# for ts in entry_times:
for ts in clean_entry_times:
    fig.add_vline(x=ts, line=dict(color="green", width=2, dash="dash"),layer="above")    
    
    
# # for ts in exit_times:
for ts in clean_exit_times:
    fig.add_vline(x=ts, line=dict(color="red", width=2, dash="dash"),layer="above")
    
# ── FINAL LAYOUT ──────────────────────────────────────────────────────────────
fig.update_layout(
    title=f"{TICKER} Intraday {start_date}",
    height=700, width=1000,
    barmode="overlay",
    xaxis_rangeslider_visible=False,
    margin=dict(l=50,r=20,t=80,b=40),
    paper_bgcolor="white", plot_bgcolor="white",
    legend=dict(orientation="h", y=-0.15, x=0.5, xanchor="center")
)

# grids & axis titles
fig.update_xaxes(showgrid=True, gridcolor="lightgray")
fig.update_yaxes(showgrid=True, gridcolor="lightgray")
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="RSI",   row=2, col=1, range=[0,100])
fig.update_yaxes(title_text="MACD",  row=3, col=1)
fig.update_yaxes(title_text="Volume",row=4, col=1)


fig.show()

,close,EMA200
529,178.97,178.994773


In [ ]:
# AVGO 
# trye: EMA200==SMA200
# EMA5<EMA9 (big sacle)
# MACD and Signal <0
# price cross belw SMA200 or EMA200 (in 30 sec) below SMA 50 or EMA 50 in 1 min
# AMZ
#goog



# plt: 
# 1- vwap > SMA(200)
# 2- SMA 50 > SMA200(50) 
# 3- EMA 50 > EMA200(50) 
# --------------------------------
# 2-MACD_Hist is rising 
# 3- RSI is rising 
# --------------------------------
# 5- EMA(50)>SMA(50)
# 6- EMA(200)>SMA(200)
# 7- macd hist>0

# if i remove atr , i might get more point
# sell point:
# big macd is deacreasing

# put threshold on sma50>sma200 or vawp>sma20

# When EMA > SMA, it generally means:Recent prices are higher than the average trend.
# Why?


adx = talib.ADX(df["high"], df["low"], df["close"], timeperiod=14)
sma20 = df["close"].rolling(20).mean()
dev = (df["close"] - sma20).abs()

df["Sideways"] = (
    (adx < 20) &
    (dev < 0.01 * df["close"]) &
    (df["MACD_hist"].abs() < 0.5)
)

# check goog buy
# if slow paste EMA 200, If fast, EMA 40